In [ ]:
from google.colab import drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive'
print(f"Drive mounted at: {base_path}")

In [ ]:
import subprocess, os, time
import shutil
import logging
from datetime import datetime
import getpass

# Global variables
DOWNLOAD_DIR = "/content/downloads"
base_path = '/content/drive/MyDrive'
LOG_FILE = "/content/magnetflow.log"

# Ensure the log file exists and is writable
with open(LOG_FILE, 'a') as f:
    pass

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

def log(msg):
    print(msg)
    logging.info(msg)

# --- Ensure external utilities are installed and available ---
# Check and install aria2c
if not os.path.exists('/usr/bin/aria2c'):
    log("WARNING: '/usr/bin/aria2c' not found. Attempting to install aria2...")
    try:
        # Use !apt-get for shell command in Colab environment
        !apt-get install -y aria2 > /dev/null
        if os.path.exists('/usr/bin/aria2c'):
            log("aria2 installed successfully.")
        else:
            log("ERROR: aria2c still not found after installation attempt. Cannot proceed.")
            raise FileNotFoundError("aria2c executable not found or installed incorrectly.")
    except Exception as e:
        log(f"ERROR: Failed during aria2 installation attempt: {e}")
        raise

# Check and install zip
if not os.path.exists('/usr/bin/zip'): # Common path for zip
    log("WARNING: 'zip' utility not found. Attempting to install 'zip'...")
    try:
        !apt-get install -y zip > /dev/null
        if os.path.exists('/usr/bin/zip'):
            log("'zip' utility installed successfully.")
        else:
            log("ERROR: 'zip' utility still not found after installation attempt. Cannot proceed with ZIP encryption.")
    except Exception as e:
        log(f"ERROR: Failed during 'zip' installation attempt: {e}")

# Check and install 7z (from p7zip-full)
if not os.path.exists('/usr/bin/7z'): # Common path for 7z from p7zip-full
    log("WARNING: '7z' utility not found. Attempting to install 'p7zip-full'...")
    try:
        !apt-get install -y p7zip-full > /dev/null
        if os.path.exists('/usr/bin/7z'):
            log("'p7zip-full' installed successfully.")
        else:
            log("ERROR: '7z' utility still not found after installation attempt. Cannot proceed with 7z encryption.")
    except Exception as e:
        log(f"ERROR: Failed during 'p7zip-full' installation attempt: {e}")
# -------------------------------------------------------------------

def baixar_magnet(magnet, tentativas=3):
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)

    for tentativa in range(1, tentativas + 1):
      log(f"⬇️ Tentativa {tentativa}: {magnet[:60]}")

      cmd = [
          "/usr/bin/aria2c", # Changed to full path to aria2c
          "--seed-time=0",
          "--summary-interval=1",
          "--allow-overwrite=true",
          "-d", DOWNLOAD_DIR,
          magnet
      ]

      process = subprocess.Popen(
          cmd,
          stdout=subprocess.PIPE,
          stderr=subprocess.STDOUT,
          text=True
      )

      for linha in process.stdout:
          print(linha.strip())

      code = process.wait()

      if code == 0:
          log("✅ Download concluído")
          return True
      else:
          log("⚠️ Falha no download, tentando novamente...")
          time.sleep(5)

    log("❌ Download falhou após várias tentativas")
    return False

def encrypt_directory(download_dir, password, output_filename=None, compression_type='zip'):
    if not os.path.exists(download_dir):
        log(f"❌ Erro: Diretório de downloads '{download_dir}' não encontrado.")
        return None

    if not password:
        log("❌ Erro: Senha não pode ser vazia para criptografia.")
        return None

    if output_filename is None:
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        if compression_type == 'zip':
            output_filename = f'{timestamp}.zip'
        elif compression_type == '7z':
            output_filename = f'{timestamp}.7z'
        else:
            log(f"❌ Erro: Tipo de compressão '{compression_type}' não suportado. Use 'zip' ou '7z'.")
            return None

    output_archive_path = os.path.join("/content", output_filename)

    log(f"🔒 Criptografando diretório '{download_dir}' para '{output_archive_path}' com {compression_type.upper()}...")

    cmd = []
    if compression_type == 'zip':
        cmd = [
            "zip",
            "-r",
            "-j", # Added -j flag to junk paths
            "-P", password,
            output_archive_path,
            download_dir
        ]
    elif compression_type == '7z':
        # a: add to archive
        # -p{password}: set password
        # -mhe: encrypt header (file names and other metadata)
        # -r: recurse subdirectories
        cmd = [
            "7z",
            "a",
            f"-p{password}",
            "-mhe", # Encrypt archive header
            "-r",
            output_archive_path,
            os.path.join(download_dir, "*") # Archive contents of the directory
        ]
    else:
        log(f"❌ Erro interno: Tipo de compressão '{compression_type}' não suportado.")
        return None

    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        log(f"✅ Criptografia {compression_type.upper()} concluída: {output_archive_path}")
        # print("stdout:", result.stdout)
        # print("stderr:", result.stderr)
        return output_archive_path
    except subprocess.CalledProcessError as e:
        log(f"❌ Erro ao criptografar diretório com {compression_type.upper()}: {e}")
        log(f"stdout: {e.stdout.strip()}")
        log(f"stderr: {e.stderr.strip()}")
        return None
    except FileNotFoundError:
        log(f"❌ Erro: O comando '{compression_type}' não foi encontrado. Certifique-se de que o utilitário está instalado.")
        return None

def upload_cloud(destination_subfolder, source_path=DOWNLOAD_DIR, tentativas=3):
    # Construct the full destination path in Google Drive
    full_gdrive_path = os.path.join(base_path, destination_subfolder)
    os.makedirs(full_gdrive_path, exist_ok=True)

    for tentativa in range(1, tentativas + 1):
        log(f"☁️ Upload tentativa {tentativa} de '{source_path}' para '{full_gdrive_path}'")
        try:
            if os.path.isfile(source_path): # If it's a file (e.g., the encrypted zip)
                # Move the file to the destination folder
                shutil.move(source_path, full_gdrive_path)
                log(f"✅ Arquivo '{os.path.basename(source_path)}' movido com sucesso.")

            elif os.path.isdir(source_path): # If it's a directory (e.g., DOWNLOAD_DIR)
                # Iterate and move each item from the download directory
                for item_name in os.listdir(source_path):
                    source_item_path = os.path.join(source_path, item_name)
                    destination_item_path = os.path.join(full_gdrive_path, item_name)
                    if os.path.isfile(source_item_path):
                        shutil.move(source_item_path, destination_item_path)
                        log(f"✅ Arquivo '{item_name}' movido com sucesso.")
                    elif os.path.isdir(source_item_path):
                        # If it's a subdirectory, move the entire directory
                        shutil.move(source_item_path, destination_item_path)
                        log(f"✅ Diretório '{item_name}' movido com sucesso.")
            else:
                log(f"❌ Erro: '{source_path}' não é um arquivo nem um diretório válido para upload.")
                return False

            log(f"✅ Upload de '{source_path}' concluído")
            return True
        except Exception as e:
            log(f"⚠️ Erro durante o upload: {e}, tentando novamente...")
            time.sleep(5)

    log(f"❌ Upload de '{source_path}' falhou após várias tentativas")
    return False

def cleanup_local_downloads(zip_file_path=None):
    log(f"🗑️ Limpando diretório local: {DOWNLOAD_DIR}")
    try:
        shutil.rmtree(DOWNLOAD_DIR)
        os.makedirs(DOWNLOAD_DIR, exist_ok=True) # Recreate the directory for future downloads
        log("✅ Limpeza do diretório de downloads concluída.")
    except Exception as e:
        log(f"❌ Erro ao limpar o diretório de downloads: {e}")

    if zip_file_path and os.path.exists(zip_file_path):
        log(f"🗑️ Removendo arquivo zip temporário: {zip_file_path}")
        try:
            os.remove(zip_file_path)
            log("✅ Remoção do arquivo zip temporário concluída.")
        except Exception as e:
            log(f"❌ Erro ao remover o arquivo zip temporário: {e}")

def executar():
    print("Cole os magnets (linha vazia para terminar):")
    magnets = []
    while True:
        m = input()
        if not m:
            break
        magnets.append(m)

    destino_subfolder = input("Pasta Destino no Google Drive (ex: Torrents): ")

    # Variable to store the path of the encrypted zip, if created
    encrypted_zip_path = None

    try:
        # Perform downloads
        for magnet in magnets:
            ok = baixar_magnet(magnet)
            if not ok:
                log(f"⏭️ Pulando magnet: {magnet[:60]}")

        # Check if there are any files downloaded before attempting encryption/upload
        if not os.listdir(DOWNLOAD_DIR):
            log("ℹ️ Nenhum arquivo baixado para criptografar ou fazer upload.")
            log("🎉 Processo finalizado")
            return

        # Ask for encryption preference
        encrypt_choice = input("Deseja criptografar os downloads antes do upload? (s/n): ").lower()
        if encrypt_choice == 's':
            compression_type_choice = input("Escolha o tipo de compressão (zip ou 7z): ").lower()
            if compression_type_choice not in ['zip', '7z']:
                log("❌ Tipo de compressão inválido. Usando 'zip' por padrão.")
                compression_type_choice = 'zip'

            password_attempts = 0
            while password_attempts < 3:
                # Use getpass for hidden input
                password = getpass.getpass("Digite a senha para a criptografia do arquivo: ")
                confirm_password = getpass.getpass("Confirme a senha: ")
                if password == confirm_password:
                    if password:
                        encrypted_zip_path = encrypt_directory(DOWNLOAD_DIR, password, compression_type=compression_type_choice)
                        if encrypted_zip_path:
                            log(f"🔒 Downloads criptografados com sucesso em: {encrypted_zip_path}")
                        else:
                            log("❌ Falha na criptografia. Uploading diretório original.")
                    else:
                        log("ℹ️ Nenhuma senha fornecida. Uploading diretório original sem criptografia.")
                    break # Exit loop if passwords match or no password is provided
                else:
                    password_attempts += 1
                    if password_attempts < 3:
                        log(f"❌ As senhas não coincidem. Tente novamente. Tentativas restantes: {3 - password_attempts}")
                    else:
                        log("❌ As senhas não coincidem após 3 tentativas. Criptografia abortada.")
                        encrypted_zip_path = None # Ensure encryption is skipped

        # Determine source path for upload
        source_for_upload = encrypted_zip_path if encrypted_zip_path else DOWNLOAD_DIR

        # Perform upload
        upload_success = upload_cloud(destino_subfolder, source_for_upload)

        if not upload_success:
            log("⚠️ Upload falhou, mantendo arquivos locais.")

    finally:
        # Clean up local downloads and temporary zip if created, regardless of upload success
        cleanup_local_downloads(encrypted_zip_path)
        log("🎉 Processo finalizado")

executar()